<a href="https://colab.research.google.com/github/Huntac/gnn_example/blob/main/walkthroughs/cora_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install numpy
!pip install tensorflow
!pip install spektral

import numpy as np
import tensorflow as tf
import spektral

In [26]:
data = spektral.datasets.Cora()

adj = data[0].a
features = data[0].x
labels = data[0].y

adj = adj.todense() + np.eye(adj.shape[0])

adj = adj.astype('float32')
features = features.astype('float32')

print(features.shape)
print(adj.shape)
print(labels.shape)

train_mask = data.mask_tr
val_mask = data.mask_va
test_mask = data.mask_te

print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))

(2708, 1433)
(2708, 2708)
(2708, 7)
140
500
1000


/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [36]:
def masked_softmax_cross_entropy(logits, labels, mask):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels)
  mask = tf.cast(mask, dtype=tf.float32)
  mask /= tf.reduce_mean(mask)
  loss *= mask
  return tf.reduce_mean(loss)

def masked_accuracy(logits, labels, mask):
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
  accuracy_all = tf.cast(correct_prediction, tf.float32)
  mask = tf.cast(mask, dtype = tf.float32)
  mask /= tf.reduce_mean(mask)
  accuracy_all *= mask
  return tf.reduce_mean(accuracy_all)

In [34]:
def gnn(fts, adj, transform, activation):
  seq_fts = transform(fts)
  ret_fts = tf.matmul(adj, seq_fts)
  return activation(ret_fts)


In [32]:
def train_cora(fts, adj, gnn_fn, units, epochs, lr):
  lyr_1 = tf.keras.layers.Dense(units)
  lyr_2 = tf.keras.layers.Dense(7)

  def cora_gnn(fts, adj):
    hidden = gnn_fn(fts, adj, lyr_1, tf.nn.relu)
    logits = gnn_fn(hidden, adj, lyr_2, tf.identity)
    return logits

  optimizer = tf.keras.optimizers.Adam(learning_rate = lr)

  best_accuracy = 0.0
  for ep in range(epochs + 1):
    with tf.GradientTape() as t:
      logits = cora_gnn(fts, adj)
      loss = masked_softmax_cross_entropy(logits, labels, train_mask)

    variables = t.watched_variables()
    grads = t.gradient(loss, variables)
    optimizer.apply_gradients(zip(grads, variables))

    logits = cora_gnn(fts,adj)
    val_accuracy = masked_accuracy(logits, labels, val_mask)
    test_accuracy = masked_accuracy(logits, labels, test_mask)

    if val_accuracy > best_accuracy:
      best_accuracy = val_accuracy
      # save model in for real scenario
      # print statistics for convenience
      print('Epoch', ep, '| Training loss:', loss.numpy(), '| Val accuracy:',
            val_accuracy.numpy(), '| Test accuracy:', test_accuracy.numpy())

In [37]:
train_cora(features, adj, gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 4.685637 | Val accuracy: 0.18199998 | Test accuracy: 0.164
Epoch 1 | Training loss: 7.08173 | Val accuracy: 0.48599997 | Test accuracy: 0.524
Epoch 3 | Training loss: 3.4098866 | Val accuracy: 0.56 | Test accuracy: 0.57899994
Epoch 4 | Training loss: 2.613023 | Val accuracy: 0.60800004 | Test accuracy: 0.63500005
Epoch 5 | Training loss: 1.3992269 | Val accuracy: 0.662 | Test accuracy: 0.705
Epoch 10 | Training loss: 0.26780266 | Val accuracy: 0.67 | Test accuracy: 0.678
Epoch 12 | Training loss: 0.18319902 | Val accuracy: 0.684 | Test accuracy: 0.686
Epoch 19 | Training loss: 0.06608303 | Val accuracy: 0.686 | Test accuracy: 0.703
Epoch 23 | Training loss: 0.0349244 | Val accuracy: 0.688 | Test accuracy: 0.71199995
Epoch 24 | Training loss: 0.031801715 | Val accuracy: 0.692 | Test accuracy: 0.714
Epoch 28 | Training loss: 0.022176035 | Val accuracy: 0.69399995 | Test accuracy: 0.715
Epoch 29 | Training loss: 0.020347217 | Val accuracy: 0.698 | Test accuracy: 0

In [38]:
train_cora(features, tf.eye(adj.shape[0]), gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 1.9528365 | Val accuracy: 0.24199998 | Test accuracy: 0.25100002
Epoch 1 | Training loss: 1.6834337 | Val accuracy: 0.31999996 | Test accuracy: 0.329
Epoch 2 | Training loss: 1.4503951 | Val accuracy: 0.35999998 | Test accuracy: 0.371
Epoch 3 | Training loss: 1.2081176 | Val accuracy: 0.41199997 | Test accuracy: 0.411
Epoch 4 | Training loss: 0.96334016 | Val accuracy: 0.438 | Test accuracy: 0.439
Epoch 5 | Training loss: 0.7344038 | Val accuracy: 0.466 | Test accuracy: 0.47
Epoch 6 | Training loss: 0.5389805 | Val accuracy: 0.494 | Test accuracy: 0.48199993
Epoch 7 | Training loss: 0.38659588 | Val accuracy: 0.516 | Test accuracy: 0.5
Epoch 8 | Training loss: 0.27534932 | Val accuracy: 0.53999996 | Test accuracy: 0.509
Epoch 10 | Training loss: 0.14479953 | Val accuracy: 0.546 | Test accuracy: 0.523
Epoch 11 | Training loss: 0.10853559 | Val accuracy: 0.552 | Test accuracy: 0.527
Epoch 12 | Training loss: 0.08304457 | Val accuracy: 0.554 | Test accuracy: 0.528

In [39]:
deg = tf.reduce_sum(adj, axis = -1)
train_cora(features, adj/deg, gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 1.9440632 | Val accuracy: 0.566 | Test accuracy: 0.596
Epoch 1 | Training loss: 1.7503469 | Val accuracy: 0.67399997 | Test accuracy: 0.66
Epoch 3 | Training loss: 1.3000072 | Val accuracy: 0.70399994 | Test accuracy: 0.713
Epoch 4 | Training loss: 1.0744447 | Val accuracy: 0.73200005 | Test accuracy: 0.747
Epoch 5 | Training loss: 0.8721776 | Val accuracy: 0.7559999 | Test accuracy: 0.7709999
Epoch 6 | Training loss: 0.7028123 | Val accuracy: 0.77199996 | Test accuracy: 0.7879999
Epoch 7 | Training loss: 0.56225026 | Val accuracy: 0.78 | Test accuracy: 0.7969999
Epoch 8 | Training loss: 0.44639075 | Val accuracy: 0.786 | Test accuracy: 0.7999999
Epoch 9 | Training loss: 0.35206503 | Val accuracy: 0.78999996 | Test accuracy: 0.7999999


In [40]:
norm_deg = tf.linalg.diag(1.0/tf.sqrt(deg))
norm_adj = tf.matmul(norm_deg, tf.matmul(adj, norm_deg))
train_cora(features, norm_adj, gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 1.9531704 | Val accuracy: 0.48799998 | Test accuracy: 0.5
Epoch 1 | Training loss: 1.7892886 | Val accuracy: 0.546 | Test accuracy: 0.56700003
Epoch 3 | Training loss: 1.3902137 | Val accuracy: 0.56799996 | Test accuracy: 0.594
Epoch 4 | Training loss: 1.196192 | Val accuracy: 0.672 | Test accuracy: 0.699
Epoch 5 | Training loss: 1.0080124 | Val accuracy: 0.75 | Test accuracy: 0.76699984
Epoch 6 | Training loss: 0.8337019 | Val accuracy: 0.78 | Test accuracy: 0.79099995
Epoch 9 | Training loss: 0.42928982 | Val accuracy: 0.786 | Test accuracy: 0.79099995
Epoch 11 | Training loss: 0.26089212 | Val accuracy: 0.78800005 | Test accuracy: 0.78999996
Epoch 16 | Training loss: 0.07079962 | Val accuracy: 0.78999996 | Test accuracy: 0.79799986
